# GPT-SoVITS v2 - Vietnamese Voice Cloning Pipeline (Optimized)

Notebook này đã được tối ưu hóa để linh hoạt và dễ sử dụng hơn.

### 🚀 Quy trình thực hiện:
1. **Cài đặt môi trường**: Chạy 1 lần đầu tiên.
2. **Cấu hình chung**: Đặt tên model (Experiment Name) tại đây.
3. **Tải Dữ liệu**: Upload file âm thanh từ máy hoặc Drive.
4. **Xử lý Dữ liệu**: Cắt âm thanh và tạo phụ đề (ASR).
5. **WebUI**: Mở giao diện để Train và Inference.

In [ ]:
# @title 1. Cài đặt Môi trường, UVR5 & GPT-SoVITS (Fix Download)
# @markdown Chạy cell này để cài đặt toàn bộ thư viện và tải model thủ công.

import os
from google.colab import drive

# Mount Drive để dùng Smart Cache (tự động lưu/tải model)
if not os.path.exists('/content/drive'):
    try:
        drive.mount('/content/drive')
    except: 
        print('Skipping Drive mount (manual or local runtime)')
import sys
from IPython.display import clear_output

# 1. Clone Repo GPT-SoVITS
if not os.path.exists("GPT-SoVITS-Vietnamese"):
    print("Dang clone repository GPT-SoVITS...")
    !git clone https://github.com/tqtuan8788-ai/GPT-SoVITS-Vietnamese.git
    %cd GPT-SoVITS-Vietnamese
else:
    %cd GPT-SoVITS-Vietnamese
    !git pull

# 2. Install Dependencies
print("Dang cai dat thu vien chinh...")
!pip install -q -r requirements.txt
!pip install -q openai-whisper

# 3. Cài đặt UVR5 (để tách nhạc/vocal)
print("Dang cai dat UVR5 va cac cong cu xu ly am thanh...")
!pip install -q soundfile librosa noise-reduction onnxruntime-gpu
# Tải model UVR5
!mkdir -p uvr5_weights
!wget -nc -P uvr5_weights https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5_only_main_vocal.pth
!wget -nc -P uvr5_weights https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/onnx_dereverb_By_FoxJoy/vocals.onnx

# 4. Tải Pretrained Models THỦ CÔNG (Fix lỗi script download)
print("Dang tai pre-trained models thu cong tu Hugging Face...")

# Tạo đúng cấu trúc thư mục mà code yêu cầu: GPT_SoVITS/pretrained_models/...
base_model_dir = "GPT_SoVITS/pretrained_models"
os.makedirs(base_model_dir, exist_ok=True)
os.makedirs(f"{base_model_dir}/chinese-roberta-wwm-ext-large", exist_ok=True)
os.makedirs(f"{base_model_dir}/chinese-hubert-base", exist_ok=True)

# Tải Chinese-Roberta (Bắt buộc cho Text encoding)
!wget -nc -P {base_model_dir}/chinese-roberta-wwm-ext-large https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/config.json
!wget -nc -P {base_model_dir}/chinese-roberta-wwm-ext-large https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/pytorch_model.bin
!wget -nc -P {base_model_dir}/chinese-roberta-wwm-ext-large https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/tokenizer.json

# Tải Chinese-Hubert (Bắt buộc cho Audio encoding)
!wget -nc -P {base_model_dir}/chinese-hubert-base https://huggingface.co/TencentGameMate/chinese-hubert-base/resolve/main/config.json
!wget -nc -P {base_model_dir}/chinese-hubert-base https://huggingface.co/TencentGameMate/chinese-hubert-base/resolve/main/pytorch_model.bin
!wget -nc -P {base_model_dir}/chinese-hubert-base https://huggingface.co/TencentGameMate/chinese-hubert-base/resolve/main/preprocessor_config.json

# Tải GPT-SoVITS v2 Pretrained Models (s2G, s2D, s1bert)
!wget -nc -P {base_model_dir} https://huggingface.co/lj1995/GPT-SoVITS/resolve/main/s2G488k.pth
!wget -nc -P {base_model_dir} https://huggingface.co/lj1995/GPT-SoVITS/resolve/main/s2D488k.pth
!wget -nc -P {base_model_dir} https://huggingface.co/lj1995/GPT-SoVITS/resolve/main/s1bert25Hz-2kh-longer-epoch=68e-step=50232.ckpt

# Tải G2PW Model (Cho tiếng Trung, nhưng script vẫn check)
!wget -nc -P GPT_SoVITS/text/G2PWModel https://paddlespeech.bj.bcebos.com/Parakeet/op_g2p_model/G2PWModel/G2PWModel_1.1.zip
!unzip -o -q GPT_SoVITS/text/G2PWModel/G2PWModel_1.1.zip -d GPT_SoVITS/text/G2PWModel

clear_output()
print("✅ Cài đặt hoàn tất! Đã tải đủ models thủ công.")

In [ ]:
# @title ⚙️ 2. Cấu hình Chung (QUAN TRỌNG)
# @markdown Nhập tên experiment của bạn ở đây. Tên này sẽ được dùng xuyên suốt notebook.
exp_name = "Giong_Doc_Sach_01" # @param {type:"string"}

# @markdown Tích chọn nếu muốn tải lại model cũ từ Drive để train tiếp (Resume Training)
resume_from_drive = False # @param {type:"boolean"}

import os
from google.colab import drive

# Common paths
drive_root = "/content/drive/MyDrive/GPT_SoVITS_Models"
dataset_root = "/content/dataset"
output_root = "/content/GPT-SoVITS-Vietnamese/output"

# Mount Drive check
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

print(f"✅ Đã cấu hình: {exp_name}")

# Logic Resume
if resume_from_drive:
    drive_exp_path = os.path.join(drive_root, exp_name)
    if os.path.exists(drive_exp_path):
        print(f"🔄 Tìm thấy model cũ trong Drive. Đang khôi phục...")
        # Restore weights
        !cp -r "{drive_exp_path}"/*.pth GPT_SoVITS/weights/ 2>/dev/null || true
        !cp -r "{drive_exp_path}"/*.ckpt GPT_SoVITS/weights/ 2>/dev/null || true
        # Restore logs (checkpoints)
        !mkdir -p logs/{exp_name}
        !cp -r "{drive_exp_path}"/* logs/{exp_name}/ 2>/dev/null || true
        print("✅ Đã khôi phục model thành công!")
    else:
        print(f"⚠️ Không tìm thấy folder {exp_name} trong Drive ({drive_root}). Bỏ qua bước resume.")


In [ ]:
# @title 3. Tải Dữ liệu Audio
# @markdown Chọn nguồn dữ liệu và upload file. Tự động dùng `exp_name` đã cấu hình ở trên.

import os
from google.colab import files
import shutil

# Use global exp_name
if 'exp_name' not in globals():
    exp_name = "Giong_Doc_Sach_01" # Fallback
    print("⚠️ Warning: exp_name chưa được định nghĩa. Dùng mặc định.")

source_type = "Direct Upload" # @param ["Direct Upload", "Google Drive"]
google_drive_path = "/content/drive/MyDrive/my_audio_folder" # @param {type:"string"}

input_audio_folder = f"{dataset_root}/{exp_name}"
os.makedirs(input_audio_folder, exist_ok=True)

if source_type == "Google Drive":
    if os.path.exists(google_drive_path):
        print(f"Dang copy tu {google_drive_path}...")
        for f in os.listdir(google_drive_path):
            if f.endswith((".wav", ".mp3", ".flac")):
                shutil.copy(os.path.join(google_drive_path, f), input_audio_folder)
        print("✅ Da copy xong!")
    else:
        print("❌ Khong tim thay thu muc Google Drive!")

else:
    print("📂 Vui lòng chọn file audio (.wav, .mp3) để upload:")
    uploaded = files.upload()
    for filename in uploaded.keys():
        shutil.move(filename, os.path.join(input_audio_folder, filename))
    print("✅ Upload hoàn tất!")

# Kiểm tra kết quả
if os.path.exists(input_audio_folder):
    files = os.listdir(input_audio_folder)
    print(f"\n📁 Thư mục dataset: {input_audio_folder}")
    print(f"🎵 Số lượng file: {len(files)}")


In [ ]:
# @title 4. Cắt Audio + Tạo Phụ Đề Tiếng Việt (ASR)
# @markdown Tự động cắt và tạo file list dựa trên `exp_name`.

import os
import subprocess

# Đảm bảo đang ở đúng thư mục repo
%cd /content/GPT-SoVITS-Vietnamese

# --- FIX LỖI FFMPEG (Quan trọng) ---
print("🛠️ Đang vá lỗi đường dẫn FFMPEG...")
# Thay thế đường dẫn Windows cứng bằng 'ffmpeg' chuẩn của Linux
!sed -i 's|cmd=\[r"C:.*ffmpeg-win-x86_64-v7.1.exe", "-nostdin"\]|cmd=["ffmpeg", "-nostdin"]|g' tools/my_utils.py

# Cài đặt thêm dependencies cần thiết
!pip install -q faster-whisper

# Set PYTHONPATH
import sys
sys.path.insert(0, '/content/GPT-SoVITS-Vietnamese')
os.environ['PYTHONPATH'] = '/content/GPT-SoVITS-Vietnamese'

# Use global exp_name
if 'exp_name' not in globals():
    exp_name = "Giong_Doc_Sach_01"

input_audio_folder = f"/content/dataset/{exp_name}"
sliced_output_folder = "output/slicer_opt"
asr_output_file = f"output/{exp_name}.list"

os.makedirs(sliced_output_folder, exist_ok=True)
os.makedirs("output", exist_ok=True)

# ========== BƯỚC 1: CẮT AUDIO ==========
print("="*60)
print("🔪 BƯỚC 1: CẮT AUDIO THÀNH CÁC ĐOẠN NGẮN")
print("="*60)

# CHECK INPUT DATA (New debug step)
if not os.path.exists(input_audio_folder):
    print(f"❌ LỖI: Thư mục input không tồn tại: {input_audio_folder}")
    print("👉 Vui lòng chạy lại Step 3 để upload audio!")
    raise FileNotFoundError("Input folder missing")

files_in_input = os.listdir(input_audio_folder)
print(f"📂 Input folder: {input_audio_folder}")
print(f"🔍 Tìm thấy {len(files_in_input)} file.")
if len(files_in_input) == 0:
    print("❌ LỖI: Thư mục input rỗng! Chưa có file audio nào.")
    raise ValueError("Input folder is empty")
else:
    print(f"   - File mẫu: {files_in_input[:3]}")

# Clean output folder before slicing to avoid mixing old data
!rm -rf {sliced_output_folder}/*

# Chạy slice audio
slice_cmd = f'python tools/slice_audio.py "{input_audio_folder}" "{sliced_output_folder}" -34 4000 300 10 500 0.9 0.25 0 1'
print(f"Running: {slice_cmd}")
os.system(slice_cmd)

if os.path.exists(sliced_output_folder):
    sliced_files = [f for f in os.listdir(sliced_output_folder) if f.endswith((".wav", ".mp3", ".flac"))]
    if len(sliced_files) == 0:
        print("\n❌ LỖI NGHIÊM TRỌNG: Quá trình cắt audio không tạo ra file nào!")
        print("👉 Nguyên nhân có thể: File input quá ngắn, bị lỗi format, hoặc tham số cắt quá gắt.")
        print("👉 Vui lòng kiểm tra lại thư mục input hoặc file audio gốc.")
        raise RuntimeError("Slicing failed - No output files")
    print(f"\n✅ Đã cắt xong! Tổng cộng {len(sliced_files)} file audio.\n")
else:
    print("\n❌ LỖI: Thư mục output chưa được tạo.\n")
    raise RuntimeError("Slicing output folder missing")

# ========== BƯỚC 2: ASR TIẾNG VIỆT ==========
print("="*60)
print("🎤 BƯỚC 2: NHẬN DẠNG GIỌNG NÓI (ASR) TIẾNG VIỆT")
print("="*60)
print("⏳ Đang chạy Whisper large-v3... Vui lòng chờ 2-10 phút.\n")

# Chạy ASR
asr_cmd = f'python tools/asr/fasterwhisper_asr.py -i "{sliced_output_folder}" -o "output" -s large-v3 -l vi -p float32'
print(f"Running: {asr_cmd}")
os.system(asr_cmd)

# Rename generated file to match exp_name
generated_asr_file = "output/slicer_opt.list"
if os.path.exists(generated_asr_file):
    if os.path.exists(asr_output_file):
        os.remove(asr_output_file)
    os.rename(generated_asr_file, asr_output_file)

# Kiểm tra file .list
if os.path.exists(asr_output_file):
    with open(asr_output_file, "r", encoding="utf-8") as f:
        lines = f.readlines()
    print(f"\n✅ ĐÃ TẠO FILE .LIST THÀNH CÔNG!")
    print(f"📄 Đường dẫn: {os.path.abspath(asr_output_file)}")
    print(f"📝 Tổng số dòng: {len(lines)}")
else:
    print("❌ LỖI: Không tạo được file .list!")

print("\n" + "="*60)
print("✅ HOÀN TẤT! Tiếp tục chạy Cell 5 để mở WebUI.")
print(f"👉 Khi vào WebUI, điền đường dẫn này vào ô Text Label File: {os.path.abspath(asr_output_file)}")


In [ ]:
# @title 5. Khởi động WebUI
# @markdown **Cell này sẽ chạy liên tục.** Làm việc trên link Gradio hiện ra.

import os

%cd /content/GPT-SoVITS-Vietnamese

# Patch config paths
cwd = os.getcwd()
possible_bert_paths = [
    os.path.join(cwd, "GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large"),
    os.path.join(cwd, "pretrained_models/chinese-roberta-wwm-ext-large"),
    "/content/GPT-SoVITS/GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large"
]
found_bert = next((p for p in possible_bert_paths if os.path.exists(p)), possible_bert_paths[0])
found_hubert = found_bert.replace("chinese-roberta-wwm-ext-large", "chinese-hubert-base")

os.environ["cnhubert_base_path"] = found_hubert
os.environ["bert_path"] = found_bert
os.environ["colab_active"] = "1"
os.environ["is_share"] = "True"

print("🚀 Đang khởi động WebUI...")
print("📋 HƯỚNG DẪN:")
print("1. Tab 1 -> 1A: Điền tên experiment và đường dẫn file .list (đã hiện ở bước trên)")
print("2. Bấm Format data")
print("3. Tab 1 -> 1B: Train SoVITS & Train GPT")
print("4. Tab 1 -> 1C: Chọn model và Inference")

!sed -i "s/is_share = False/is_share = True/g" config.py
!python webui.py --share --i18n_dir ./i18n/vi_VN.json

In [ ]:
# @title (Tùy chọn) Chạy lại ASR thủ công
# @markdown Chỉ chạy cell này nếu bạn muốn chạy lại bước tạo phụ đề cho file audio đã cắt.

import os
if 'exp_name' not in globals():
    exp_name = "Giong_Doc_Sach_01"

sliced_audio_folder = "output/slicer_opt"
output_list_file = f"output/{exp_name}.list"

if not os.path.exists(sliced_audio_folder):
    print("❌ LỖI: Không tìm thấy thư mục đã cắt!")
else:
    print("🚀 Đang chạy Whisper (large-v3)...")
    !python tools/asr/fasterwhisper_asr.py -i "{sliced_audio_folder}" -o "output" -s large-v3 -l vi -p float32
    
    # Rename
    generated = "output/slicer_opt.list"
    if os.path.exists(generated):
        import shutil
        shutil.move(generated, output_list_file)
    
    print(f"✅ Đã xong: {os.path.abspath(output_list_file)}")

In [ ]:
# @title 6. Lưu Model về Drive
save_to_drive = True # @param {type:"boolean"}

if 'exp_name' not in globals():
    exp_name = "Giong_Doc_Sach_01"

drive_save_path = f"/content/drive/MyDrive/GPT_SoVITS_Models/{exp_name}"

if save_to_drive:
    from google.colab import drive
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')
    
    os.makedirs(drive_save_path, exist_ok=True)
    print(f"Dang luu model {exp_name} vao Drive...")

    !cp GPT_SoVITS/pretrained_models/s2G*.pth "{drive_save_path}"/
    !cp GPT_SoVITS/pretrained_models/s1bert*.ckpt "{drive_save_path}"/
    !cp -r GPT_SoVITS/weights/* "{drive_save_path}"/ 2>/dev/null || true
    !cp -r logs/{exp_name}/* "{drive_save_path}"/ 2>/dev/null || true
    
    print(f"✅ Da luu xong tai: {drive_save_path}")